In [137]:
import os
import torch
import pandas as pd
from torch import nn
from models.part_cem import PartCEM
import matplotlib.pyplot as plt
from datasets import build_datasets
from torchvision.utils import make_grid
import torch.nn.functional as F
from torchvision.models import resnet50, ResNet

In [146]:
b,k,c,h,w = 16, 8, 128, 14, 14
a = torch.randn(b, k, c)
torch.mean(torch.sub(torch.ones(16,8,8), torch.eye(8)))

tensor(0.8750)

In [119]:
b,k,c,h,w = 16, 8, 128, 14, 14

maps = torch.randn(b, k, h, w)

loss_pres = torch.nn.functional.avg_pool2d(maps[:, :, 2:-2, 2:-2], 3, stride=1).max(-1)[0].max(-1)[0].max(0)[0].mean()
loss_pres

tensor(1.1266)

In [ ]:
def pres_loss(maps: torch.Tensor):
    maps = maps[..., 2:-2, 2:-2]
    maps = F.avg_pool2d(maps, 3, stride=1)
    max_vals = F.max_pool2d(maps, 8) # shape: [b, k, 1, 1]
    batch_precs, _ = max_vals.max(0) # shape: [k, 1, 1]
    return batch_precs.mean()
    ...

In [132]:
F.max_pool2d(F.avg_pool2d(maps[:, :, 2:-2, 2:-2], 3, stride=1), 8).max(0)[0].mean()

tensor(1.1266)

In [136]:
F.max_pool2d(F.avg_pool2d(maps[:, :, 2:-2, 2:-2], 3, stride=1), 8).shape

torch.Size([16, 8, 1, 1])

In [130]:
F.max_pool2d(torch.randn(4,3,3), 3, 1)

tensor([[[1.0247]],

        [[2.7199]],

        [[1.8444]],

        [[1.7397]]])